# Predicting Functionality of Waterpoints in Tanzania

![img](images/lake_victoria.jpg)

Lake Victoria in Tanzania - photo courtesy of [___]().

# Overview

# Business Problem

About 4 million of Tanzania's 59 million residents lack access to potable (drinking) water; an even greater proportion of the Tanzanian population (nearly half) lack access to what water.org calls "[improved sanitation](https://water.org/our-impact/where-we-work/tanzania/)".

While the majority of Tanzanians *do* have access to clean water, a quick look at the functionality status of about 60,000 waterpoints (wells, pumps, etc.) indicates that nearly **half** of those waterpoints either need repair to function consistently.

![img](images/target_val_counts.jpg)

# Data

Data used in this classification project comes from an ongoing competition hosted by DrivenData, [*Pump it Up: Data Mining the Water Table*](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/).

Repository structure for running this notebook available at the bottom of the [README.md](https://github.com/toastdeini/dsc-ph3-Tanzanian-water-well-status/blob/main/README.md).

Descriptions of each column can be found at [this link](data_dict_basic.txt) within this repository.

## Loadout

In [2]:
# Packages for data cleaning, plotting, and manipulation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# machine learning libraries/functions/classes

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression, RidgeCV
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, \
                             StackingClassifier, ExtraTreesClassifier
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier

In [3]:
# Importing training data
train_val = pd.read_csv('data/training_set_values.csv')

# Only using `status_group` column from label set, to
# avoid duplicating `id` column
train_label = pd.read_csv('data/training_set_labels.csv',
                             usecols = ['status_group'])


# Test set, provided by DrivenData - not to be used
# until models have been trained, internally validated, etc.

test_df = pd.read_csv('data/test_set_values.csv')

In [4]:
# Concatenating separate .csv files
df = pd.concat(objs = [train_val, train_label],
               axis = 1)

In [5]:
# Quick readout
df.sample(n = 5,
          random_state = 138)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
27487,60684,0.0,2013-02-15,Dwe,1137,DWE,37.136702,-4.075092,Madukani,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,non functional
52964,457,0.0,2011-04-06,NaN,0,NaN,34.368774,-8.767996,Kwa Mzee Lusambo,0,...,soft,good,seasonal,seasonal,river,river/lake,surface,communal standpipe,communal standpipe,non functional
26478,7855,50.0,2011-03-24,Private Individual,-8,Da,38.991011,-6.537616,Bakari,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59113,50299,0.0,2013-03-19,Government Of Tanzania,1068,DWE,36.806314,-3.448869,Kwa Elishirikiamwea Swai,0,...,soft,good,enough,enough,spring,spring,groundwater,other,other,functional
14153,70382,0.0,2011-07-27,He,0,HE,31.635313,-1.718094,Kabubuya,0,...,soft,good,enough,enough,spring,spring,groundwater,hand pump,hand pump,non functional


## Feature Comprehension and Selection

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

We have about **forty** potential features here, indicated by columns `0` through `39`; column `40`, `status_group`, is our target variable. The majority (30) of those columns are currently stored as type `object`, the remainder as either `int64` or `float64`.

Thorough exploratory analysis revealed that a number of these columns contained information that was redundant with other columns, not informative to the modeling process, or otherwise superfluous.

In [7]:
# Dropping columns determined to be either irrelevant or
# superfluous in exploratory analysis

cols_to_drop = [
    'id',  # unique identifier, not useful for modeling
    'date_recorded',  # superfluous information, too many unique records
    'recorded_by',  # no unique information + no unique values across 59.4k rows
    'funder',   
    'installer',  # large number of unique values (see also `funder`);
                  # may be added back in later
    'wpt_name',  # identifier column, not useful for modeling
    'num_private',  # data dict. does not provide details for this column
    'subvillage',  # too many unique values - uninformative for modeling
    'region_code',  # redundant information vis-a-visa the simpler `region`
    'district_code',  # may be added back in later
    'lga',
    'ward',  # redundant location data (with `lga`)
    'scheme_management',  # may be added back in later
    'scheme_name',  # large number of nulls, redundant vis-a-vis `scheme_management`
    'extraction_type',
    'extraction_type_group',  # using `extraction_type_class` for generalized info
    'management',
    'management_group',  # may be added back in later
    'payment',  # identical information to `payment_type`
    'water_quality',  # comparable information to `quality_group` - redundant
    'quantity_group',  # identical information to `quantity` - redundant
    'source',  # redundant with other `source_` columns
    'waterpoint_type'  # used `waterpoint_type_group` instead
]

In [8]:
df = df.drop(columns = cols_to_drop).copy()

# Modeling

## Preprocessing

### Pipelines and Transformers

In [9]:
# Subpipes for imputing median values - to be used for `latitude` and `longitude`
subpipe_lat      = Pipeline(steps=[('num_impute', SimpleImputer(missing_values = -2.000000e-08,
                                                                strategy = 'median')),
                                   ('ss', StandardScaler())])

subpipe_long     = Pipeline(steps=[('num_impute', SimpleImputer(missing_values = 0.000000,
                                                                strategy = 'median')),
                                   ('ss', StandardScaler())])


# Subpipe for imputing median values
subpipe_num      = Pipeline(steps=[('num_impute', SimpleImputer(strategy = 'median')),
                                   ('ss', StandardScaler())])

# Subpipe for `construction_year`
subpipe_year     = Pipeline(steps=[('num_impute', SimpleImputer(missing_values = 0,
                                                                strategy = 'median')),
                                   ('ss', StandardScaler())])

# Subpipe for categorical features including `basin`, `payment_type`
subpipe_cat      = Pipeline(steps=[('freq_imputer_nan', SimpleImputer(strategy = 'most_frequent')),
                                   ('freq_imputer_unk', SimpleImputer(strategy = 'most_frequent',
                                                                      missing_values = 'unknown')),
                                   ('ohe', OneHotEncoder(drop = 'if_binary',
                                                         sparse = False,
                                                         handle_unknown = 'ignore'))])

In [10]:
# Columns to be passed through numerical pipeline
num_cols = ['amount_tsh',
            'gps_height',
            'population']

# Columns to be passed through categorical pipeline
cat_cols = ['basin',
            'region',
            'payment_type',
            'quantity',
            'quality_group',
            'permit',
            'public_meeting',
            'extraction_type_class',
            'source_type',
            'source_class',
            'waterpoint_type_group']

In [11]:
ct = ColumnTransformer(transformers = [
    ('subpipe_num', subpipe_num, num_cols),
    ('subpipe_year', subpipe_year, ['construction_year']),
    ('subpipe_long', subpipe_long, ['longitude']),
    ('subpipe_lat', subpipe_lat, ['latitude']),
    ('subpipe_cat', subpipe_cat, cat_cols)],
                       remainder = 'passthrough')

# ('subpipe_label', subpipe_label, ['status_group'])

### Train/Test Split

In [12]:
# Splitting DataFrame into features/values DataFrame
# (i.e. `X`) and labels series (`y`)

X = df.drop('status_group', axis = 1)
y = df['status_group']

In [13]:
# Splitting internal training data into separate
# training and test sets for (eventual) internal validation

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 138)

## Iterative Modeling

Description of models.

Explanation of results of models.

## Final Model

Description of model.

Explanation of results of model.

### Accuracy on Unseen Data

# Conclusions

## Next Steps

# References/Other